## Практическая работа

В этой практической работе четыре обязательные задачи.

*Обязательные задачи* нужно сделать для того, чтобы проверить, что вы действительно усвоили материал модуля. Сдайте их на проверку.

Удачи!

Цели практической работы: 


1.   Потренироваться в обучении моделей деревьев решений.
2.   Потренироваться в обучении моделей случайного леса. 
3.   Научиться оценивать качество моделей с помошью Accuracy и confusion matrix.
4.   Научиться увеличивать качество моделей с помощью тюнинга параметров.




Что оценивается:

*   Все пункты и критерия приёмки задания выполнены.
*   Удаление колонок по результатам feature_importance и определения типов производится с помощью кода, а не перечислением их вручную.




## Обязательные задачи

### Описание датасета:
- `id`: идентификатор записи;
- `is_manufacturer_name`: признак производителя автомобиля;

- `region_*`: регион;
- `x0_*`: тип топлива;
- `manufacturer_*`: производитель;
- `short_model_*`: сокращённая модель автомобиля;
- `title_status_*`: статус;
- `transmission_*`: коробка передач;
- `state_*`: штат;
- `age_category_*`: возрастная категория автомобиля;

- `std_scaled_odometer`: количество пройденных миль (после стандартизации);
- `year_std`: год выпуска (после стандартизации);
- `lat_std`: широта (после стандартизации);
- `long_std`: долгота (после стандартизации);
- `odometer/price_std`: отношение стоимости к пробегу автомобиля (после стандартизации);
- `desc_len_std`: количество символов в тексте объявления о продаже (после стандартизации);
- `model_in_desc_std`: количество наименований модели автомобиля в тексте объявления о продаже (после стандартизации);
- `model_len_std`: длина наименования автомобиля (после стандартизации);
- `model_word_count_std`: количество слов в наименовании автомобиля (после стандартизации);
- `month_std`: номер месяца размещения объявления о продаже автомобиля (после стандартизации);
- `dayofweek_std`: день недели размещения объявления о продаже автомобиля (после стандартизации);
- `diff_years_std`: количество лет между годом производства автомобиля и годом размещения объявления о продаже автомобиля (после стандартизации);

- `price`: стоимость;
- `price_category`: категория цены.

0. *Подготовка базовой модели*

Обучите простую модель классификации с помощью DecisionTreeClassifier на данных из датасета vehicles_dataset_prepared.csv. Для этого сделайте следующие шаги:

1. Обучите модель дерева решений с зафиксированным random_state на тренировочной выборке.
2. Сделайте предикт на тестовой выборке.
3. Замерьте точность на тестовой выборке и выведите матрицу ошибок. 
4. Удалите фичи с нулевыми весами по feature_importance из тренировочной и тестовой выборок.
5. Заново обучите модель и измерьте качество.

In [44]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score, confusion_matrix 

In [75]:
df = pd.read_csv('vehicles_dataset_prepared.csv')

df_prepared = df.copy()
df_prepared = df_prepared.drop(['price', 'odometer/price_std'], axis=1)

x = df_prepared.drop(['price_category'], axis=1)
y = df_prepared['price_category']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [76]:
dtc = DecisionTreeClassifier(random_state=4)
dtc.fit(x_train, y_train)

DecisionTreeClassifier(random_state=4)

In [77]:
y_pred = dtc.predict(x_test)
print(accuracy_score(y_test, y_pred))

0.6812196812196812


In [78]:
confusion_matrix(y_test, y_pred)

array([[748,  49, 200],
       [ 50, 692, 211],
       [192, 218, 526]], dtype=int64)

In [79]:
ft_imp = sorted(list(zip(dtc.feature_names_in_, dtc.feature_importances_)), key=lambda x: x[1], reverse=True)
ft_imp = [feature for feature, importance in ft_imp if importance > 0.0]
ft_imp[:20]

['age_category_new',
 'std_scaled_odometer',
 'model_len_std',
 'desc_len_std',
 'diff_years_std',
 'lat_std',
 'long_std',
 'id',
 'x0_diesel',
 'year_std',
 'dayofweek_std',
 'short_model_wrangler',
 'manufacturer_ram',
 'manufacturer_kia',
 'short_model_f-150',
 'short_model_silverado',
 'manufacturer_hyundai',
 'manufacturer_gmc',
 'manufacturer_mercedes-benz',
 'short_model_Scion']

In [80]:
x_train = x_train[ft_imp]
x_test = x_test[ft_imp]

dtc = DecisionTreeClassifier(random_state=4)
dtc.fit(x_train, y_train)

y_pred = dtc.predict(x_test)
print(accuracy_score(y_test, y_pred))

0.6690921690921691


In [81]:
confusion_matrix(y_test, y_pred)

array([[734,  54, 209],
       [ 41, 677, 235],
       [190, 226, 520]], dtype=int64)

1. *Подготовка модели случайного леса*

Обучите простую модель классификации с помощью RandomForestClassifier. Для этого сделайте следующие шаги:
1. На новых урезанных семплах тренировочной и тестовой выборок обучите модель случайного леса с зафиксированным random_state=50. 

2. Сделайте предикт и посчитайте точность модели и матрицу ошибок. Сравните с предыдущей моделью дерева решений. Есть ли случаи, когда модель из пункта 1 отрабатывает лучше, чем модель случайного леса?

In [52]:
from sklearn.ensemble import RandomForestClassifier


rfc = RandomForestClassifier(random_state=50)
rfc.fit(x_train, y_train)

RandomForestClassifier(random_state=50)

In [53]:
y_pred = rfc.predict(x_test)
accuracy_score(y_test, y_pred)

0.7529452529452529

In [54]:
confusion_matrix(y_test, y_pred)

array([[811,  33, 153],
       [ 17, 782, 154],
       [157, 199, 580]], dtype=int64)

2. *Тюнинг модели случайного леса*

Увеличьте точность модели на тестовом датасете RandomForestClassifier c помощью тюнинга параметров. 

Параметры, которые можно настраивать для увеличения точности:

```
    `bootstrap'
    'max_depth'
    'max_features'
    'min_samples_leaf'
    'min_samples_split'
    'random_state'
    'n_estimators'

```



С описанием каждого из параметров можно ознакомиться в документации:


https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

Задание засчитывается, если значение метрики строго выше 0,76 на тестовом датасете.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV


rfc = RandomForestClassifier()

param_grid = {
    'bootstrap': [True, False],
    'max_depth': list(range(8, 33, 8)),
    'max_features': ['sqrt', 'log2'],
    'min_samples_leaf': list(range(1, 5)),
    'min_samples_split': list(range(2, 5, 2)),
    'random_state': [4, 42, 50],
    'n_estimators': list(range(100, 501, 100))
}

randomized_search_rfс = RandomizedSearchCV(
    estimator=rfc,
    param_distributions=param_grid,
    scoring='accuracy',
    n_iter=20,
    verbose=1,
    n_jobs=-1
)

randomized_search_rfс.fit(x_train, y_train)

best_params = randomized_search_rfс.best_params_
best_params

Fitting 5 folds for each of 20 candidates, totalling 100 fits


{'random_state': 50,
 'n_estimators': 300,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 32,
 'bootstrap': False}

In [ ]:
"""
{'random_state': 50,
 'n_estimators': 300,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 32,
 'bootstrap': False}
"""

rfc_tuned = RandomForestClassifier(**best_params)
rfc_tuned.fit(x_train, y_train)

y_pred = rfc_tuned.predict(x_test)
accuracy_score(y_test, y_pred)

0.761954261954262

In [57]:
confusion_matrix(y_test, y_pred)

array([[781,  44, 172],
       [ 15, 808, 130],
       [173, 215, 548]], dtype=int64)

3. *Анализ влияния фичей на модель*

Во всех задачах до вы работали над подготовленным датасетом, где фичи были заранее извлечены из текстовых переменных, отскейлены и пропущены через One Hot Encoder. Сравним, какой была бы предсказательная способность модели, если бы мы использовали только сырые данные из исходного датасета. Для этого проделайте следующие шаги:

1. Загрузите датасет `vehicles_dataset_old.csv`.
2. Удалите из него переменную `price` и все строковые колонки. Дерево решений и случайный лес не умеют самостоятельно работать со строковыми значениями.
3. Сформируйте x_train и x_test так же, как они были сформированы в предыдущих заданиях.
4. Обучите свою лучшую модель случайного леса на новых данных и замерьте качество. Убедитесь, что оно ухудшилось.
5. Найдите три фичи, которые лучшим образом повлияли на предсказательную способность модели.

In [86]:
df_old = pd.read_csv('vehicles_dataset_old.csv')
df_old.head()

,id,url,region,region_url,price,year,manufacturer,model,fuel,odometer,title_status,transmission,image_url,description,state,lat,long,posting_date,price_category,date
0,7308295377,https://chattanooga.craigslist.org/ctd/d/chatt...,chattanooga,https://chattanooga.craigslist.org,54990,2020,ram,2500 crew cab big horn,diesel,27442,clean,other,https://images.craigslist.org/00N0N_1xMPvfxRAI...,Carvana is the safer way to buy a car During t...,tn,35.060000,-85.250000,2021-04-17T12:30:50-0400,high,2021-04-17 16:30:50+00:00
1,7316380095,https://newjersey.craigslist.org/ctd/d/carlsta...,north jersey,https://newjersey.craigslist.org,16942,2016,ford,explorer 4wd 4dr xlt,other,60023,clean,automatic,https://images.craigslist.org/00x0x_26jl9F0cnL...,***Call Us for more information at: 201-635-14...,nj,40.821805,-74.061962,2021-05-03T15:40:21-0400,medium,2021-05-03 19:40:21+00:00
2,7313733749,https://reno.craigslist.org/ctd/d/atlanta-2017...,reno / tahoe,https://reno.craigslist.org,35590,2017,volkswagen,golf r hatchback,gas,14048,clean,other,https://images.craigslist.org/00y0y_eeZjWeiSfb...,Carvana is the safer way to buy a car During t...,ca,33.779214,-84.411811,2021-04-28T03:52:20-0700,high,2021-04-28 10:52:20+00:00
3,7308210929,https://fayetteville.craigslist.org/ctd/d/rale...,fayetteville,https://fayetteville.craigslist.org,14500,2013,toyota,rav4,gas,117291,clean,automatic,https://images.craigslist.org/00606_iGe5iXidib...,2013 Toyota RAV4 XLE 4dr SUV Offered by: R...,nc,35.715954,-78.655304,2021-04-17T10:08:57-0400,medium,2021-04-17 14:08:57+00:00
4,7303797340,https://knoxville.craigslist.org/ctd/d/knoxvil...,knoxville,https://knoxville.craigslist.org,14590,2012,bmw,1 series 128i coupe 2d,other,80465,clean,other,https://images.craigslist.org/00F0F_5UAXmOzC18...,Carvana is the safer way to buy a car During t...,tn,35.970000,-83.940000,2021-04-08T15:10:56-0400,medium,2021-04-08 19:10:56+00:00


In [87]:
df_old = df_old.drop('price', axis=1)
df_old = df_old[df_old.select_dtypes(include='number').columns.to_list() + ['price_category']]
df_old

,id,year,odometer,lat,long,price_category
0,7308295377,2020,27442,35.060000,-85.250000,high
1,7316380095,2016,60023,40.821805,-74.061962,medium
2,7313733749,2017,14048,33.779214,-84.411811,high
3,7308210929,2013,117291,35.715954,-78.655304,medium
4,7303797340,2012,80465,35.970000,-83.940000,medium
...,...,...,...,...,...,...
9614,7304876387,2002,150000,42.123900,-79.189500,low
9615,7316152972,2008,113573,43.216990,-77.755610,medium
9616,7310993818,2011,150184,44.925908,-122.982753,low
9617,7306637427,2015,61943,43.029559,-89.397796,high


In [88]:
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix


x_train, x_test, y_train, y_test = train_test_split(df_old.drop('price_category', axis=1), df_old['price_category'], test_size=0.3, random_state=42)
x_train

,id,year,odometer,lat,long
3781,7314842398,2019,24551,61.569390,-149.356450
6101,7316005173,1995,42000,37.986400,-122.135000
491,7316501929,2014,57923,42.260000,-71.800000
3017,7307760453,2019,32791,47.199100,-122.315100
3872,7314536873,1995,177000,33.930800,-81.147300
...,...,...,...,...,...
5734,7316686098,2019,15392,40.697349,-74.219903
5191,7306406566,2002,197000,42.230805,-85.546932
5390,7306000351,2008,153152,33.836000,-79.048000
860,7307147953,2019,12102,32.190000,-80.740000


In [89]:
rfc_old = RandomForestClassifier(**best_params)
rfc_old.fit(x_train, y_train)

y_pred = rfc_old.predict(x_test)
accuracy_score(y_test, y_pred)

0.6278586278586279

In [90]:
confusion_matrix(y_test, y_pred)

array([[695,  76, 226],
       [ 47, 690, 216],
       [264, 245, 427]], dtype=int64)

In [91]:
ft_imp = sorted(list(zip(rfc_old.feature_names_in_, rfc_old.feature_importances_)), key=lambda x: x[1], reverse=True)
ft_imp = [feature for feature, importance in ft_imp if importance > 0.0]
ft_imp[:3]

['odometer', 'year', 'id']